# GATEWAY CLASS USAGE 


Instead of using directly the Orderbook class, you can use this class to simulate a real historical trading session and introduce your own orders in between while considering the latency effect of the market data and market access

In [1]:
import os
print(os.getcwd())

/home/paco/dev/PythonMatchingEngine/examples


In [2]:
import timeit
import time
import datetime
import numpy as np
from marketsimulator.gateway import Gateway


"""
 Create a new Gateway instance with Acciona historical orders.
 This creates a new orderbook and fills it with the first orders to 
 reconstruct the orderbook present when the orderbook opened that day.
 
 It is like market replay of what happened that day and you can mix your own
 orders in between.
 
 The class Gateway instanciates an Orderbook object (Gateway.ob) 
 and provides methods  to move the historical 
 replay forward tick by tick or a number of seconds 
 
 The expected total latency is configured to be 2e4microsecs==20ms 
 
"""
gtw = Gateway(ticker='san',
             date=datetime.date(2019,5,23),
             start_h=9,
             end_h=10,
             latency=20_000)

# Market orderbook right after the opening auction
print(f'orderbook opened at {gtw.ob_time} \n')
# Market Orderbook is initialized so that it shows the orderbook 
# that was present right after the opening auction took place
print(gtw.ob)



orderbook opened at 2019-05-23 09:00:31.127000 

     vbid    pbid    pask   vask
0   12682  4.0255  4.0265   6907
1    6896  4.0250  4.0275   4000
2    2500  4.0240  4.0285   6907
3    1355  4.0230  4.0310  12992
4   17850  4.0210  4.0340  72965
5    7000  4.0205  4.0365  16625
6  104505  4.0200  4.0390    999
7     248  4.0190  4.0400  32265
8    5000  4.0170  4.0410  13442
9      60  4.0160  4.0450  15550


In [3]:
# Move the orderbook 1 minute forward in time
gtw.move_n_seconds(60)
# Check orderbook time
print(f'New orderbook time is {gtw.ob_time} \n')
# Check new orderbook
print(gtw.ob)


New orderbook time is 2019-05-23 09:01:31.127000 

    vbid    pbid    pask   vask
0   1187  4.0165  4.0200   4491
1   2500  4.0160  4.0210   5219
2  25000  4.0150  4.0220   2719
3   5219  4.0135  4.0230   5219
4    500  4.0130  4.0240   4691
5   2719  4.0125  4.0250   5219
6   8300  4.0120  4.0260   2666
7   5219  4.0115  4.0265    253
8  21189  4.0110  4.0270  12666
9   8214  4.0105  4.0275   5491


## QUERYING MARKET DATA

#### PUBLIC MARKET DATA

In [4]:
# Get first 3 best bids and asks with price and volume
print(gtw.ob.top_bids(3))
print(gtw.ob.top_asks(3))

[[4.0165, 4.016, 4.015], [1187, 2500, 25000]]
[[4.02, 4.021, 4.022], [4491, 5219, 2719]]


In [5]:
# If you don't need the volume, it is faster to ask just for the prices
print(gtw.ob.top_bidpx(3))
print(gtw.ob.top_askpx(3))

[4.0165, 4.016, 4.015]
[4.02, 4.021, 4.022]


This is the time it takes for top_bids

In [6]:
%%timeit
gtw.ob.top_bids(3)

2.62 µs ± 38.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Just querying the price is faster since the volume accum in a price level
is just calculated upon request to speed things up

In [7]:
%%timeit
gtw.ob.top_bidpx(3)

1.66 µs ± 56.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The fastest access to market data is querying for the best bid and best ask only:

In [8]:
print(gtw.ob.bbid)
print(gtw.ob.bask)

(4.0165, 1187)
(4.02, 4491)


In [9]:
%%timeit 
gtw.ob.bbid

357 ns ± 4.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
# Everytime there is a matching this updates
# Market trades vol
print(gtw.ob.trades_vol)
print('')
# Market trades prices 
print(gtw.ob.trades_px)
print('')
# Market trades timestamp
print(gtw.ob.trades_time)


[ 2485.   223.    50.   500.   750.  2481.   407.  2500.  1000.   355.
  1050.  5700.   800.   400.  2050.  1609.   479.  1412.  1588.  2012.
   750.  1038.   650.  1854.  1158.  1038.  1875.   650.  2593.   966.
  3458.  3542.  3712.  1240.  1100. 10000.  6000.    76.   124.  1000.
   500.  1000.  1000.  2000.  2486.   120.   475.  2490.   500.   330.
  3700.  2000.  5000.   374.  2000.  3900.   300.    74.   373.  3200.
  2000.   100.  3725.   250.  4000.  2000.  1000.  2487.  2000.  5000.
  6000.  1000.    60.  2000.   250.    62.   500.  1100.  1030.  1000.
   340.   140.   737.   200.  1000.  2000.  1000.  1500.  1200.   940.
   745.   250.   497.   163.  1000.   112.   500.   495.  2000.  2200.
   370.   100.    90.   500.  5200.  5200.  9997.   248.   852.  1176.
   400.   500.   416.   240.  2072.  2500.   334.  1355.   852.    60.
  3219. 12994.  5200.  6806.  2000.   700.   250.  1000.  1400.  1038.
    45.    52.  1318.  2500.    93.  2719.   881.   273.   449.  2570.
   559

In [11]:
# Check orderbook cummulative traded volume
gtw.ob.cumvol

258935

#### PRIVATE DATA FEED

In [12]:
# Market trades vol
print(gtw.ob.my_trades_vol)
print('')
# Market trades prices 
print(gtw.ob.my_trades_px)
print('')
# Market trades timestamp
print(gtw.ob.my_trades_time)

[]

[]

[]


In [13]:
# Check orderbook cummulative traded volume
gtw.ob.my_cumvol

0

## LATENCY SIMULATION
In this section we are going to send our own orders to the Orderbook using Gateway methods allowing it to simulate the effect of latency in our orders when reaching the market

In [14]:
# Check best ask price to set as our target price
target_price = gtw.ob.bask[0]
mkt_time_when_target_px_showed = gtw.ob_time
print(f'We just saw ask price:{target_price}'  \
        f' that happened at time {mkt_time_when_target_px_showed}')

# Send an aggressive buy order against this ask price for inmediate execution
# We are actually queuing it with gtw.latency added 
my_uid = gtw.queue_my_new(is_buy=True,
                          qty=10,
                          price=target_price)


We just saw ask price:4.02 that happened at time 2019-05-23 09:01:31.127000


In [15]:
# Check order status.
# NOTE: time has not moved yet. Therefore,
# my order is still on the fly, it did not arrive to the orderbook
# If we check its status in the orderbook we will get a KeyError
try:
    gtw.ord_status(my_uid)
except KeyError:
    print(f'The order with uid:{my_uid} did not arrive to the orderbook')


The order with uid:-1 did not arrive to the orderbook


In [16]:
# Advance time 1 second to give the order time to arrive
gtw.move_n_seconds(1)

# Check status again. 
# I was lucky, the price I targeted did not disappear while my order was
# and flying and we got it filled (leavesqty==0) => HIT RATE 100%
# Since it is filled, it is not active anymore (active==False)
try:
    print(gtw.ord_status(my_uid))
except KeyError:
    print(f'The order with uid:{my_uid} did not arrive to the orderbook')


{'uid': -1, 'is_buy': True, 'qty': 10, 'cumqty': 10, 'leavesqty': 0, 'price': 4.02, 'timestamp': Timestamp('2019-05-23 09:01:31.147000'), 'active': False}


In [17]:
# Check my trades
print(gtw.ob.my_trades_vol)
print(gtw.ob.my_trades_px)

# As we can see, our execution happened exactly 20 miliseconds after the
# ask price we targeted appeared in the first place. Just as expected. 
print(gtw.ob.my_trades_time)
print('')
print('Our execution was done 20 ms after the price first showed')
print(gtw.ob.my_trades_time[0]-mkt_time_when_target_px_showed)


[10.]
[4.02]
[Timestamp('2019-05-23 09:01:31.147000')]

Our execution was done 20 ms after the price first showed
0 days 00:00:00.020000


### USING RELATIVE PRICING - PEGGING
Getting the correct tick size in post MiFID II european cash equity markets is a bit tricky. We provide helper functions to allow you to move up or down a number of ticks without have to worry about the propper ticksize rounding in edge cases. 

In [18]:
# Current orderbook orderbook. The spread is wide, lets close it.
print(gtw.ob)

    vbid    pbid    pask   vask
0   1187  4.0165  4.0200    754
1   2500  4.0160  4.0210   2719
2  25000  4.0150  4.0220   2719
3   5219  4.0135  4.0230   5219
4    500  4.0130  4.0240   4691
5   2719  4.0125  4.0250   5219
6   8300  4.0120  4.0260   2666
7   5219  4.0115  4.0265    253
8  21189  4.0110  4.0270  12666
9   8214  4.0105  4.0275   5491


In [19]:
# Now we are goint to send an order that improves the best bid
# by one tick, closing the orderbook bid-ask spread in one tick
bbid_px = gtw.ob.bbid[0]
# this helper function gives you a new price from a ref price and a number of ticks shift
imp_bbid = gtw.ob.get_new_price(price=bbid_px, n_moves=1)
# we send our order to the Orderbook through the Gateway
my_uid = gtw.queue_my_new(is_buy=True,
                          qty=7,
                          price=imp_bbid)

In [20]:
# Lets move gtw.latency microseconds in time to let our order arrive
gtw.move_n_seconds(gtw.latency/1e6+1)

In [21]:
# Lets check our order status
gtw.ord_status(my_uid)

{'uid': -2,
 'is_buy': True,
 'qty': 7,
 'cumqty': 0,
 'leavesqty': 7,
 'price': 4.017,
 'timestamp': Timestamp('2019-05-23 09:01:32.147000'),
 'active': True}

In [22]:
# Great, its already active and setting the new best bid
print(gtw.ob.bbid)
print()
# Lets see the new orderbook
print(gtw.ob)


(4.017, 7)

    vbid    pbid    pask   vask
0      7  4.0170  4.0200   5423
1   1187  4.0165  4.0220   2719
2   2500  4.0160  4.0230   5219
3  25000  4.0150  4.0240   4691
4   5219  4.0135  4.0250   5219
5    500  4.0130  4.0260   2666
6   2719  4.0125  4.0265    253
7   8300  4.0120  4.0270  12666
8  15219  4.0115  4.0275   4000
9  21189  4.0110  4.0280   2666


In [23]:
# Allright, I was just bluffing, I do not want to buy => lets cancel it
gtw.queue_my_cancel(my_uid)

# Move forward
gtw.move_n_seconds(gtw.latency/1e6+1)

# Lets check our order status
gtw.ord_status(my_uid)

# Lets check the book now
print(gtw.ob)

    vbid    pbid    pask   vask
0   1187  4.0165  4.0200   5423
1   2500  4.0160  4.0210   2719
2  25000  4.0150  4.0220   2719
3   5219  4.0135  4.0230   5219
4    500  4.0130  4.0240   4691
5   2719  4.0125  4.0250   5219
6   8300  4.0120  4.0260   2666
7  15219  4.0115  4.0265    253
8  21189  4.0110  4.0270  12666
9   8214  4.0105  4.0275   4000
